# Data Equivariance

The networks implemented in DeepH-pack are equivariant networks, which strictly follow the piror knowledge that the physical properties transform equivariantly under spatial translations and rotations. On the other hand, the data (e.g., Hamiltonians and density matrices) generated by the DFT programs may slightly break the equivariance rule because of the discrete mesh in the real space. Obviously, the equivariance error is part of the unphysical noise of the training data, which is the lower bound of the train loss of the neural network. So that to properly train and evaluate a DeepH model, it is crucial to keep the equivariance error of the data below the expression error of the model. DeepH-dock provides a tool chain to systematically test the data equivariance.

## Structure generation

First step to test the equivariance is generating a group of structures that differ from each other only by the translations and rotations. User can use the terminal tool to generate randomly translated and rotated structures from a given initial POSCAR file.

The command and the description of the arguments are shown below.

In [2]:
!dock analyze dft-equiv gen -h

Usage: dock analyze dft-equiv gen [OPTIONS] POSCAR_DIR

  Generate transformed structures for equivariance testing

Options:
  -n, --generate-num INTEGER RANGE
                                  The number of translated and rotated
                                  structures.  [default: 64; 1<=x<=1024]
  -r, --rot-mode [0|1|2|3]        Rotation mode: 0 - no rotation, 1 - rotate
                                  coordinates only, 2 - rotate lattice only, 3
                                  - both rotate coordinates and lattice.
                                  [default: 1]
  --translate                     Translate the structure.
  --move-to-origin                Move the mass center to origin.
  --dump-decimals INTEGER         The decimals of coordinates and lattice when
                                  dumping, -1 for no rounding.  [default: -1]
  -h, --help                      Show this message and exit.


The initial structure must be placed as <POSCAR_DIR>/0/POSCAR and other structures will be dumped as <POSCAR_DIR>/[1-n]/POSCAR.

Be careful of the `rot-mode` argument. Mode 0 denotes no rotation and is used for translational equiviance test. For periodic systems, only mode 3 is allowed for rotational equiviance test. For cluster systems with regular real space mesh (e.g., OpenMX, SIESTA, etc), only mode 1 and 2 are meaningful for rotational equiviance test. For cluster systems with atomic-centered mesh (e.g., FHI-aims), only mode 1 and 3 are meaningful for rotational equiviance test.

Example:

In [6]:
!dock analyze dft-equiv gen ./poscars -n 4 --translate
!ls -1 ./poscars/*/POSCAR

100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1219.16it/s]
./poscars/0/POSCAR
./poscars/1/POSCAR
./poscars/2/POSCAR
./poscars/3/POSCAR


## DFT calculation

Run the DFT calculations for all the structures and transform the results to the standard file format of DeepH-pack. The result of each structure should be placed under the same directory as the POSCAR file. This needs to be done by the user.

## Equivariance check

DeepH-dock provides the terminal tool to check the equivariance error of the results. The error is defined as $|H_{(\mathcal{R_0})}-D^{\dagger}_{(R_i)}H_{(\mathcal{R_i})}D_{(R_i)}|$, where $\mathcal{R_0}$ denotes the original structure, $R_i$ denotes the rotation of $i$-th structure ($\mathcal{R_i}=R_i\mathcal{R_0}$), and $D_{(R_i)}$ denotes the Wigner-D matrix related to $R_i$.

In most of the Hamiltonian training cases, the equivariance error of data needs to be lower than 0.1meV, and an error of 0.01meV is recommended.

The command and the description of the arguments are shown below.

In [ ]:
!dock analyze dft-equiv test -h

Example:

In [ ]:
!dock analyze dft-equiv test dft_calc.bak